In [1]:
import requests
from bs4 import BeautifulSoup

def extract_scoe_programs():
    url = 'https://engineering.saraswatikharghar.edu.in/programs-offered'
    try:
        res = requests.get(url, timeout=10)
        res.raise_for_status()
    except Exception as e:
        print("Error fetching SCOE:", e)
        return []

    soup = BeautifulSoup(res.text, 'html.parser')
    heading = soup.find(lambda tag: tag.name.startswith('h') and 'Programs Offered' in tag.get_text())
    table = heading.find_next('table') if heading else None
    if not table:
        return []

    programs = []
    rows = table.find_all('tr')[1:]
    for row in rows:
        cols = row.find_all('td')
        if not cols:
            continue

        raw_html = cols[0].decode_contents()
        lines = BeautifulSoup(raw_html.replace('<br>', '\n'), 'html.parser').get_text().split('\n')

        for line in lines:
            program_name = line.strip()
            if program_name:
                programs.append({
                    "university": "SCOE",
                    "location": "Kharghar, Navi Mumbai",
                    "program": program_name
                })
    return programs

def extract_vjti_programs():
    url = 'https://vjti.ac.in/undergraduate-admission/'
    try:
        res = requests.get(url, timeout=10)
        res.raise_for_status()
    except Exception as e:
        print("Error fetching VJTI:", e)
        return []

    soup = BeautifulSoup(res.text, 'html.parser')
    table = soup.find('table')
    if not table:
        return []

    programs = []
    rows = table.find_all('tr')[1:]
    for row in rows:
        cols = row.find_all('td')
        if len(cols) > 1:
            raw_html = cols[1].decode_contents()
            lines = BeautifulSoup(raw_html.replace('<br>', '\n'), 'html.parser').get_text().split('\n')
            for line in lines:
                program_name = line.strip()
                if program_name:
                    programs.append({
                        "university": "VJTI",
                        "location": "Mumbai",
                        "program": program_name
                    })
    return programs

def extract_ace_programs():
    url = 'https://acetvm.com/admission/'
    try:
        res = requests.get(url, timeout=10)
        res.raise_for_status()
    except Exception as e:
        print("Error fetching ACE:", e)
        return []

    soup = BeautifulSoup(res.text, 'html.parser')
    tables = soup.find_all('table')
    if not tables:
        return []

    rows = tables[0].find_all('tr')[1:]
    programs = []
    for row in rows:
        cols = row.find_all('td')
        if not cols:
            continue

        raw_html = cols[0].decode_contents()
        lines = BeautifulSoup(raw_html.replace('<br>', '\n'), 'html.parser').get_text().split('\n')

        for line in lines:
            program = line.strip()
            if program and not any(char.isdigit() for char in program):
                programs.append({
                    "university": "ACE College of Engineering",
                    "location": "Thiruvananthapuram, Kerala",
                    "program": program.title()
                })
    return programs

def extract_all_programs():
    all_programs = (
        extract_scoe_programs() +
        extract_vjti_programs() +
        extract_ace_programs()
    )

    print("\n📚 All Programs with University and Location:\n")
    for item in all_programs:
        print(f"• {item['program']} — {item['university']}, {item['location']}")

    return all_programs  # Optional: return the list if you want to reuse it

# Run the scraper
if __name__ == "__main__":
    extract_all_programs()



📚 All Programs with University and Location:

• Computer Engineering — SCOE, Kharghar, Navi Mumbai
• Computer Science and Engineering (Artificial Intelligence and Machine Learning) — SCOE, Kharghar, Navi Mumbai
• Computer Science and Engineering (Data Science) — SCOE, Kharghar, Navi Mumbai
• Mechanical Engineering — SCOE, Kharghar, Navi Mumbai
• Civil Engineering — SCOE, Kharghar, Navi Mumbai
• Information Technology — SCOE, Kharghar, Navi Mumbai
• Automobile Engineering — SCOE, Kharghar, Navi Mumbai
• M.E.(Civil)-Structural Engineering — SCOE, Kharghar, Navi Mumbai
• Civil Engineering — VJTI, Mumbai
• Computer Engineering — VJTI, Mumbai
• Electrical Engineering — VJTI, Mumbai
• Electronics Engineering — VJTI, Mumbai
• Electronics & Telecommunication Engineering — VJTI, Mumbai
• Information Technology — VJTI, Mumbai
• Mechanical Engineering — VJTI, Mumbai
• Production Engineering (Sandwich) — VJTI, Mumbai
• Textile Technology — VJTI, Mumbai
• Aeronautical Engineering (Ao) — ACE Colleg

In [2]:
def get_user_profile():
    print("\nEnter your academic interests (comma-separated, e.g., AI, Robotics):")
    interests = input().split(',')

    print("\nEnter your strengths (comma-separated, e.g., strong in math, good communicator):")
    strengths = input().split(',')

    print("\nEnter your weaknesses (comma-separated, e.g., weak in math, low confidence):")
    weaknesses = input().split(',')

    print("\nEnter your preferences (e.g., location like India):")
    preferences = input().split(',')

    return {
        "interests": [i.strip().lower() for i in interests],
        "strengths": [s.strip().lower() for s in strengths],
        "weaknesses":[w.strip().lower() for w in weaknesses],
        "preferences": [p.strip().lower() for p in preferences]
    }

In [3]:
def generate_opportunities_and_threats(strengths, weaknesses, api_key):
    prompt = f"""You are an academic career advisor AI. A student has the following:

Strengths: {', '.join(strengths)}
Weaknesses: {', '.join(weaknesses)}

Based on this, give:
1. Two personalized **opportunities** they can leverage in their academic or career path.
2. Two potential **threats** they should be aware of or mitigate.
"""

    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://openrouter.ai",
        "X-Title": "AI SWOT Analyzer"
    }

    data = {
        "model": "openrouter/cypher-alpha:free",
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }

    try:
        response = requests.post(url, headers=headers, json=data, timeout=30)
        response.raise_for_status()
        result = response.json()
        return result["choices"][0]["message"]["content"].strip()
    except Exception as e:
        return f"❌ Failed to generate opportunities and threats: {e}"

In [26]:
def perform_swot(profile, api_key):
    strengths = profile.get('strengths', [])
    weaknesses = profile.get('weaknesses', [])

    raw_response = generate_opportunities_and_threats(strengths, weaknesses, api_key)

    swot = {
        "Strengths": strengths,
        "Weaknesses": weaknesses,
        "Opportunities": [],
        "Threats": []
    }

    lines = raw_response.splitlines()
    current_section = None

    for line in lines:
        line = line.strip()
        if not line:
            continue

        # Detect section headers
        if line.lower().startswith("opportunities"):
            current_section = "Opportunities"
            continue
        elif line.lower().startswith("threats"):
            current_section = "Threats"
            continue

        # Bullet-style content
        if line.startswith("-") and current_section:
            swot[current_section].append(line.lstrip("- ").strip())

        # 🧠 Smart fallback: Look for threat-like phrases
        elif "lack of confidence" in line or "be cautious" in line or "prevent" in line:
            swot["Threats"].append(line)

    # Fallback notices
    if not swot["Opportunities"]:
        swot["Opportunities"].append("⚠️ No clear opportunities found in LLM response.")
    if not swot["Threats"]:
        swot["Threats"].append("⚠️ No clear threats found in LLM response.")

    return swot

In [28]:
def match_programs(profile):
    all_programs = extract_all_programs()
    matched = []

    # Normalize user input
    interests = [x.strip().lower() for x in profile['interests']]
    preferences = [x.strip().lower() for x in profile['preferences']]

    for program in all_programs:
        course_words = program['program'].lower().replace("(", "").replace(")", "").replace("&", "and").split()
        location_words = program['location'].lower().split()

        # Check if any interest is a word in course
        interest_match = any(interest in course_words for interest in interests)
        location_match = any(pref in location_words for pref in preferences)

        if interest_match and location_match:
            matched.append(program)

    if not matched:
        print("\n⚠️ No programs matched your criteria. Try keywords like 'ai', 'computer', 'engineering', or relax location.\n")

    return matched[:5]  # Return top 5 matches


In [30]:
import requests

def generate_advice(profile, recommendations, api_key):
    if not recommendations:
        return "No strong matches found. Try expanding your preferences or exploring related fields."

    # Compose user prompt
    prompt = f"""You are an academic advisor AI helping students with college program selection.

Student Interests: {', '.join(profile['interests'])}
Strengths: {', '.join(profile['strengths'])}
Preferences: {', '.join(profile['preferences'])}

Recommended Programs:
{chr(10).join([f"- {r['program']} at {r['university']} ({r['location']})" for r in recommendations])}

Please provide:
1. Two lines of personalized application advice.
2. One motivational sentence.
3. One strength the student should highlight in their SOP.
"""

    # OpenRouter API request
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://openrouter.ai",  # optional but helpful for free usage
        "X-Title": "AI Admission Counselor"
    }

    data = {
        "model": "openrouter/cypher-alpha:free",  # You can change this to another valid model
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }

    try:
        response = requests.post(url, headers=headers, json=data, timeout=30)
        response.raise_for_status()
        result = response.json()
        return result["choices"][0]["message"]["content"].strip()

    except Exception as e:
        return f"❌ OpenRouter API Error: {e}"


In [32]:
def print_results(profile, swot, recommendations, advice):
    print("\n--- SWOT Analysis ---")
    for k, v in swot.items():
        print(f"{k}: {', '.join(v)}")

    print("\n--- Recommended Programs ---")
    if recommendations:
        for r in recommendations:
            print(f"{r['university']} - {r['program']} ({r['location']})")
    else:
        print("No matching programs found.")

    print("\n--- Personalized Advice ---")
    print(advice)

In [46]:
def main():
    api_key = "your_api_key" 
    profile = get_user_profile()
    swot = perform_swot(profile, api_key)
    matched_programs = match_programs(profile)
    advice = generate_advice(profile, matched_programs, api_key)

    print_results(profile, swot, matched_programs, advice)

In [48]:
if __name__ == "__main__":
    main()


Enter your academic interests (comma-separated, e.g., AI, Robotics):


 Engineering



Enter your strengths (comma-separated, e.g., strong in math, good communicator):


 Confidence



Enter your weaknesses (comma-separated, e.g., weak in math, low confidence):


 Management



Enter your preferences (e.g., location like India):


 Mumbai



📚 All Programs with University and Location:

• Computer Engineering — SCOE, Kharghar, Navi Mumbai
• Computer Science and Engineering (Artificial Intelligence and Machine Learning) — SCOE, Kharghar, Navi Mumbai
• Computer Science and Engineering (Data Science) — SCOE, Kharghar, Navi Mumbai
• Mechanical Engineering — SCOE, Kharghar, Navi Mumbai
• Civil Engineering — SCOE, Kharghar, Navi Mumbai
• Information Technology — SCOE, Kharghar, Navi Mumbai
• Automobile Engineering — SCOE, Kharghar, Navi Mumbai
• M.E.(Civil)-Structural Engineering — SCOE, Kharghar, Navi Mumbai
• Civil Engineering — VJTI, Mumbai
• Computer Engineering — VJTI, Mumbai
• Electrical Engineering — VJTI, Mumbai
• Electronics Engineering — VJTI, Mumbai
• Electronics & Telecommunication Engineering — VJTI, Mumbai
• Information Technology — VJTI, Mumbai
• Mechanical Engineering — VJTI, Mumbai
• Production Engineering (Sandwich) — VJTI, Mumbai
• Textile Technology — VJTI, Mumbai
• Aeronautical Engineering (Ao) — ACE Colleg